In [1]:
# -*- coding: utf-8 -*-

from tensorflow import keras
from tensorflow.keras import layers

from libs import cf_m2m
# from libs import ds_layer_p2p_001

from libs import ds_layer_p2p

In [2]:
# ===========================================
# Required only on the local gpu rtx 2070 (may be due to some memory issue)
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Memory settings
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

#============================================

2024-10-14 20:41:10.743529: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:10.748295: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:10.748396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:10.898976: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9



2024-10-14 20:41:11.204919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:11.205049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:11.205145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-14 20:41:11.205248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 222 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:

# lite_cf_ds_dBI model architecture (Prototype BBAs: 4 masses)
def get_model(img_size, ldr_size, prototypes, singleton_num):
    
    inputs_img=keras.Input(shape=img_size+(3,), name='rgb')
    inputs_ldr=keras.Input(shape=ldr_size+(3,), name='lidar')
    
    #x: image processing brach
    #y: lidar processing brach
    
    # Encoder
    #B1: Block 1
    x=layers.ZeroPadding2D(padding=1, name='Block1_rgb_zp')(inputs_img)
    x=layers.Conv2D(32,4,strides=2,activation='elu', name='Block1_rgb_conv')(x)
    y=layers.ZeroPadding2D(padding=1, name='Block1_lidar_zp')(inputs_ldr)
    y=layers.Conv2D(32,4,strides=2,activation='elu', name='Block1_lidar_conv')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block1_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block1_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block1_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block1_lidar_cf')([y_fsn,y])
    #B2: Block 2
    x=layers.ZeroPadding2D(padding=1,name='Block2_rgb_zp')(x_fsn)
    x=layers.Conv2D(32,3,strides=1,activation='elu',name='Block2_rgb_conv')(x) 
    y=layers.ZeroPadding2D(padding=1,name='Block2_lidar_zp')(y_fsn)
    y=layers.Conv2D(32,3,strides=1,activation='elu', name='Block2_lidar_conv')(y) 
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block2_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block2_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block2_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block2_lidar_cf')([y_fsn,y])
    #B3: Block 3
    x=layers.ZeroPadding2D(padding=1,name='Block3_rgb_zp')(x_fsn)
    x=layers.Conv2D(64,4,strides=2,activation='elu', name='Block3_rgb_conv')(x)  
    y=layers.ZeroPadding2D(padding=1, name='Block3_lidar_zp')(y_fsn)
    y=layers.Conv2D(64,4,strides=2,activation='elu', name='Block3_lidar_conv')(y)   
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block3_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block3_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block3_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block3_lidar_cf')([y_fsn,y])
    #B4: Block 4
    x=layers.ZeroPadding2D(padding=1, name='Block4_rgb_zp')(x_fsn)
    x=layers.Conv2D(64,3,strides=1,activation='elu', name='Block4_rgb_conv')(x) 
    y=layers.ZeroPadding2D(padding=1, name='Block4_lidar_zp')(y_fsn)
    y=layers.Conv2D(64,3,strides=1,activation='elu', name='Block4_lidar_conv')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block4_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block4_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block4_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block4_lidar_cf')([y_fsn,y])
    #B5: Block 5
    x=layers.ZeroPadding2D(padding=1,name='Block5_rgb_zp')(x_fsn)
    x=layers.Conv2D(128,4,strides=2,activation='elu', name='Block5_rgb_conv')(x) 
    y=layers.ZeroPadding2D(padding=1, name='Block5_lidar_zp')(y_fsn)
    y=layers.Conv2D(128,4,strides=2,activation='elu', name='Block5_lidar_conv')(y)  
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block5_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block5_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block5_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block5_lidar_cf')([y_fsn,y])
    
    # Context module
    #B6: Block 6
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block6_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block6_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block6_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block6_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block6_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block6_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block6_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block6_lidar_cf')([y_fsn,y])
    #B7: Block 7
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block7_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block7_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block7_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block7_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block7_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block7_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block7_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block7_lidar_cf')([y_fsn,y])
    #B8: Block 8
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(2,2),activation='elu', name='Block8_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block8_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(2,2),activation='elu', name='Block8_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block8_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block8_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block8_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block8_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block8_lidar_cf')([y_fsn,y])
    #B9: Block 9
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(4,4),activation='elu', name='Block9_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block9_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(4,4),activation='elu', name='Block9_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block9_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block9_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block9_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block9_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block9_lidar_cf')([y_fsn,y])
    #B10: Block 10
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(8,8),activation='elu', name='Block10_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block10_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(8,8),activation='elu', name='Block10_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block10_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block10_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block10_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block10_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block10_lidar_cf')([y_fsn,y])
    #B11: Block 11
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(16,16),activation='elu', name='Block11_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block11_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(16,16),activation='elu', name='Block11_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block11_liar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block11_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block11_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block11_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block11_lidar_cf')([y_fsn,y])
    #B12: Block 12
    x=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block12_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block12_rgb_dp')(x)
    y=layers.Conv2D(128,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block12_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block12_lidar_dp')(y)
    # Fusion
    x_fsn=cf_m2m.c_fusion_wt(name='Block12_lidar_cfw')(y)
    x_fsn=layers.Add(name='Block12_rgb_cf')([x_fsn,x])
    y_fsn=cf_m2m.c_fusion_wt(name='Block12_rgb_cfw')(x)
    y_fsn=layers.Add(name='Block12_lidar_cf')([y_fsn,y])
    #B13: Block 13
    x=layers.Conv2D(128,1,padding="same",activation='elu', name='Block13_rgb_conv')(x_fsn)
    x=layers.Dropout(0.25, name='Block13_rgb_dp')(x)
    y=layers.Conv2D(128,1,padding="same",activation='elu', name='Block13_lidar_conv')(y_fsn)
    y=layers.Dropout(0.25, name='Block13_lidar_dp')(y)
    # Fusion
    y_wtd=cf_m2m.c_fusion_wt(name='Block13_lidar_cfw')(y)
    x_wtd=cf_m2m.c_fusion_wt(name='Block13_rgb_cfw')(x)
    ctx_out=layers.Add(name='context_output')([y_wtd,x_wtd])
    
    # Decoder
    #B14: Block 14
    x=layers.Conv2DTranspose(64,4,strides=2,activation='elu', padding='same', name='Block14_convtp')(ctx_out)
    #B15: Block 15
    x=layers.ZeroPadding2D(padding=1,name='Block15_zp')(x)
    x=layers.Conv2D(64,3,strides=1,activation='elu', name='Block15_conv')(x)
    #B16: Block 16
    x=layers.Conv2DTranspose(32,4,strides=2,activation='elu', padding='same', name='Block16_convtp')(x)
    #B17: Block 17
    x=layers.ZeroPadding2D(padding=1, name='Block17_zp')(x)
    x=layers.Conv2D(32,3,strides=1,activation='elu', name='Block17_conv')(x)
    #B18: Block 18
    feature=layers.Conv2DTranspose(8,4,strides=2, activation='elu', padding='same', name='Block18_convtp')(x)
    # Evidential formulation
    x = ds_layer_p2p.DS1(prototypes, name='distance_prototype')(feature)
    x = ds_layer_p2p.DS1_activate(name='prototype_activation')(x)
    x = ds_layer_p2p.DS2(singleton_num, name='prototype_singleton_mass')(x)
    x = ds_layer_p2p.DS2_omega(name='prototype_singleton_omega_mass')(x)
    
    # Define the model
    model=keras.Model(inputs=[inputs_img, inputs_ldr], outputs=x)
    
    return model

In [5]:
img_size = (384,1248)   # camera image size
ldr_size= (384,1248)    # lidar image size
prototypes = 8        # number of prototypes
singleton_num = 2       # number of singletons

#Build model
model = get_model(img_size, ldr_size, prototypes, singleton_num)
model.summary()
#keras.utils.plot_model(model, show_shapes=True)

# save model
model.save('model_lite_cf_ds_proto8_conflict_BBA')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lidar (InputLayer)             [(None, 384, 1248,   0           []                               
                                3)]                                                               
                                                                                                  
 rgb (InputLayer)               [(None, 384, 1248,   0           []                               
                                3)]                                                               
                                                                                                  
 Block1_lidar_zp (ZeroPadding2D  (None, 386, 1250, 3  0          ['lidar[0][0]']                  
 )                              )                                                           